In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import joblib
import sklearn

In [10]:
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("scikit-learn version:", sklearn.__version__ )
print("joblib version:", joblib.__version__)


numpy version: 1.21.5
pandas version: 1.4.2
scikit-learn version: 1.0.2
joblib version: 1.1.0


In [11]:
requirements = f"""
numpy=={np.__version__}
pandas=={pd.__version__}
scikit-learn=={sklearn.__version__}
joblib=={joblib.__version__}
"""

# Write the requirements string to a file
with open('requirements.txt', 'w') as file:
    file.write(requirements)

# Inflation Data

In [12]:
inflation_data = pd.read_excel("inflation rate data.xlsx")

In [13]:
inflation_data

,Year,Inflation Rate (%),Annual Change
0,2021.0,0.1695,0.0371
1,2020.0,0.1325,0.0185
2,2019.0,0.1140,-0.0070
3,2018.0,0.1209,-0.0443
4,2017.0,0.1652,0.0085
...,...,...,...
57,1964.0,0.0086,0.0355
58,1963.0,-0.0269,-0.0796
59,1962.0,0.0527,-0.0101
60,1961.0,0.0628,0.0083


In [14]:
inflation_data.shape

(62, 3)

# Features and Labels

In [15]:
df1=inflation_data[['Year','Inflation Rate (%)']]
df1

,Year,Inflation Rate (%)
0,2021.0,0.1695
1,2020.0,0.1325
2,2019.0,0.1140
3,2018.0,0.1209
4,2017.0,0.1652
...,...,...
57,1964.0,0.0086
58,1963.0,-0.0269
59,1962.0,0.0527
60,1961.0,0.0628


In [16]:
# Reshape Year array
X = df1['Year'].values.reshape(-1,1)
y = df1['Inflation Rate (%)'].values.reshape(-1,1)
print('features:',X,'labels:',y)

features: [[2021.]
 [2020.]
 [2019.]
 [2018.]
 [2017.]
 [2016.]
 [2015.]
 [2014.]
 [2013.]
 [2012.]
 [2011.]
 [2010.]
 [2009.]
 [2008.]
 [2007.]
 [2006.]
 [2005.]
 [2004.]
 [2003.]
 [2002.]
 [2001.]
 [2000.]
 [1999.]
 [1998.]
 [1997.]
 [1996.]
 [1995.]
 [1994.]
 [1993.]
 [1992.]
 [1991.]
 [1990.]
 [1989.]
 [1988.]
 [1987.]
 [1986.]
 [1985.]
 [1984.]
 [1983.]
 [1982.]
 [1981.]
 [1980.]
 [1979.]
 [1978.]
 [1977.]
 [1976.]
 [1975.]
 [1974.]
 [1973.]
 [1972.]
 [1971.]
 [1970.]
 [1969.]
 [1968.]
 [1967.]
 [1966.]
 [1965.]
 [1964.]
 [1963.]
 [1962.]
 [1961.]
 [1960.]] labels: [[ 0.1695]
 [ 0.1325]
 [ 0.114 ]
 [ 0.1209]
 [ 0.1652]
 [ 0.1568]
 [ 0.0901]
 [ 0.0806]
 [ 0.0848]
 [ 0.1222]
 [ 0.1084]
 [ 0.1372]
 [ 0.1256]
 [ 0.1158]
 [ 0.0539]
 [ 0.0823]
 [ 0.1786]
 [ 0.15  ]
 [ 0.1403]
 [ 0.1288]
 [ 0.1887]
 [ 0.0693]
 [ 0.0662]
 [ 0.1   ]
 [ 0.0853]
 [ 0.2927]
 [ 0.7284]
 [ 0.5703]
 [ 0.5717]
 [ 0.4459]
 [ 0.1301]
 [ 0.0736]
 [ 0.5047]
 [ 0.5451]
 [ 0.1129]
 [ 0.0572]
 [ 0.0744]
 [ 0.1782]
 [ 0.

# Online Learning Model

In [17]:
class OnlineLearningModel:
    def __init__(self):
        self.sgd = SGDRegressor(loss='squared_loss', penalty='l2', random_state=0)
        self.scaler = StandardScaler()
        self.batch_size = 100
        self.n_iterations = 10
        self.scaler_fit = False

    def train(self, X, y):
        self.scaler.fit(X)
        self.scaler_fit = True

        total_samples = len(X)
        for _ in range(self.n_iterations):
            indices = np.random.permutation(total_samples)
            batches = [indices[i:i + self.batch_size] for i in range(0, total_samples, self.batch_size)]

            for batch in batches:
                X_batch = X[batch]
                y_batch = y[batch]
                X_batch_scaled = self.scaler.transform(X_batch)
                self.sgd.partial_fit(X_batch_scaled, y_batch)

    def process_new_data(self, X_new, y_new):
        if self.scaler_fit:
            self.scaler.partial_fit(X_new)
        else:
            self.scaler.fit(X_new)
            self.scaler_fit = True

        total_samples = len(X_new)
        for _ in range(self.n_iterations):
            indices = np.random.permutation(total_samples)
            batches = [indices[i:i + self.batch_size] for i in range(0, total_samples, self.batch_size)]

            for batch in batches:
                X_batch = np.array(X_new)[batch]
                y_batch = np.array(y_new)[batch]
                X_batch_scaled = self.scaler.transform(X_batch)
                self.sgd.partial_fit(X_batch_scaled, y_batch)

    def predict(self, year):
        X_new_scaled = self.scaler.transform([[year]])
        y_pred = self.sgd.predict(X_new_scaled)
        y_pred_rounded = np.round((y_pred), 4)  
        return y_pred_rounded

    def save_model(self, filepath):
        joblib.dump(self, filepath)

    @staticmethod
    def load_model(filepath):
        return joblib.load(filepath)

# Ofline Training 

In [18]:
model = OnlineLearningModel()
model.train(X, y)  

C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

# Online Training

In [9]:

inflation_years = input("Enter the years (separated by commas): ")
inflation_rates = input("Enter the inflation rates (separated by commas): ")

# Split the input years and rates into separate lists
years = [int(year.strip()) for year in inflation_years.split(',')]
rates = [float(rate.strip()) for rate in inflation_rates.split(',')]

for year, rate in zip(years, rates):
    model.process_new_data([[year]], [[rate]])



# Prediction

In [19]:
model.predict(2099)

array([0.2275])

# Saving Model

In [20]:
model.save_model("model.pkl")

# Prediction from Save Model

In [21]:
loaded_model = OnlineLearningModel.load_model("model.pkl")

loaded_model.predict(2022)


array([0.1565])

# Exchange Data

In [22]:
exchange_data = pd.read_excel('exchange rate.xlsx')

In [23]:
exchange_data

,Rate Date,Currency,Rate Year,Rate Month,Buying Rate,Central Rate,Selling Rate
0,5/25/2023,US DOLLAR,2023.0,May,460.4500,460.9500,461.4500
1,5/25/2023,POUNDS STERLING,2023.0,May,569.7148,570.3334,570.9521
2,5/25/2023,EURO,2023.0,May,494.3391,494.8759,495.4127
3,5/25/2023,SWISS FRANC,2023.0,May,509.1784,509.7313,510.2842
4,5/25/2023,YEN,2023.0,May,3.3038,3.3074,3.3110
...,...,...,...,...,...,...,...
52222,2001-10-12 00:00:00,CFA,2001.0,December,0.1514,0.1521,0.1528
52223,2001-10-12 00:00:00,EURO,2001.0,December,99.3787,99.8230,100.2670
52224,2001-10-12 00:00:00,WAUA,2001.0,December,138.8876,139.5084,140.1293
52225,2001-10-12 00:00:00,US DOLLAR,2001.0,December,111.8500,112.3500,112.8500


In [24]:
exchange_data.shape


(52227, 7)

# Filter Data

In [25]:
exchange_data = exchange_data[exchange_data['Currency'] == 'US DOLLAR']
exchange_data

,Rate Date,Currency,Rate Year,Rate Month,Buying Rate,Central Rate,Selling Rate
0,5/25/2023,US DOLLAR,2023.0,May,460.45,460.95,461.45
12,5/24/2023,US DOLLAR,2023.0,May,460.31,460.81,461.31
24,5/23/2023,US DOLLAR,2023.0,May,460.31,460.81,461.31
36,5/22/2023,US DOLLAR,2023.0,May,460.28,460.78,461.28
48,5/19/2023,US DOLLAR,2023.0,May,460.31,460.81,461.31
...,...,...,...,...,...,...,...
52201,12/14/2001,US DOLLAR,2001.0,December,112.10,112.60,113.10
52204,12/13/2001,US DOLLAR,2001.0,December,111.85,112.35,112.85
52213,2001-12-12 00:00:00,US DOLLAR,2001.0,December,111.85,112.35,112.85
52216,2001-11-12 00:00:00,US DOLLAR,2001.0,December,111.85,112.35,112.85


# Features and Labels

In [26]:
df=exchange_data[['Rate Year','Central Rate']]
df

,Rate Year,Central Rate
0,2023.0,460.95
12,2023.0,460.81
24,2023.0,460.81
36,2023.0,460.78
48,2023.0,460.81
...,...,...
52201,2001.0,112.60
52204,2001.0,112.35
52213,2001.0,112.35
52216,2001.0,112.35


In [27]:
# Reshape Year array
X = df['Rate Year'].values.reshape(-1,1)
y = df['Central Rate'].values.reshape(-1,1)
print('features:',X,'labels:',y)


features: [[2023.]
 [2023.]
 [2023.]
 ...
 [2001.]
 [2001.]
 [2001.]] labels: [[460.95]
 [460.81]
 [460.81]
 ...
 [112.35]
 [112.35]
 [112.35]]


# Training

In [28]:
exchange_model = OnlineLearningModel()
exchange_model.train(X, y)  

C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

# Online Training

In [20]:

exchange_years = input("Enter the years (separated by commas): ")
currency_rates = input("Enter the inflation rates (separated by commas): ")

# Split the input years and rates into separate lists
years = [int(year.strip()) for year in exchange_years.split(',')]
rates = [float(rate.strip()) for rate in currency_rates.split(',')]

for year, rate in zip(years, rates):
    exchange_model.process_new_data([[year]], [[rate]])


Enter the years (separated by commas): 2012
Enter the inflation rates (separated by commas): 300.01


C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\hasna\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

# Prediction

In [21]:
exchange_model.predict(2023)

array([371.9027])

# Saving Model

In [22]:
# Save the model to a file
exchange_model.save_model("exchange_model.pkl")


# Load Model

In [23]:
# Load the saved model
Exchange_loaded_model = OnlineLearningModel.load_model("exchange_model.pkl")

# Use the loaded model for prediction
Exchange_loaded_model.predict(2035)


array([550.1752])

# Prediction

In [24]:
def calculate_value(naira_amount, years, future=True):
    if not future and years > 23:
        print("Please enter a number of years less than or equal to 23 for the past value calculation.")
        return None
    
    # Project inflation rate and exchange rate for the given number of years
    projected_year = 2023 + years if future else 2023 - years
    projected_inflation_rate = max(0, loaded_model.predict(projected_year))
    projected_exchange_rate = max(0, Exchange_loaded_model.predict(projected_year))
    
    current_exchange_rate = exchange_data[exchange_data['Rate Year'] == 2023]['Central Rate'].values[0]
    
    if future:
        # Calculate future value
        future_value = naira_amount * (1 + projected_inflation_rate)**years * (projected_exchange_rate / current_exchange_rate)
        future_value = np.round((future_value), 2)
        return future_value
    else:
        # Calculate past value
        past_value = naira_amount / ((1 + projected_inflation_rate)**years * (projected_exchange_rate / current_exchange_rate))
        past_value = np.round((past_value), 2)
        return past_value



In [ ]:

def show_menu():
    print("Choose one of the following options:")
    print("A. Inflation Learning")
    print("B. Exchange Learning")
    print("C. Calculation")
    print("M. Go back to menu")
    print("E. Exit")


loaded_model = OnlineLearningModel.load_model("model.pkl")
Exchange_loaded_model = OnlineLearningModel.load_model("exchange_model.pkl")

while True:
    show_menu()
    session_choice = input("Enter your choice: ")

    if session_choice.upper() == 'A':
        inflation_years = input("Enter the years (separated by commas): ")
        inflation_rates = input("Enter the inflation rates (separated by commas): ")

        years = [int(year.strip()) for year in inflation_years.split(',')]
        rates = [float(rate.strip()) for rate in inflation_rates.split(',')]

        for year, rate in zip(years, rates):
            loaded_model.process_new_data([[year]], [[rate]])
        loaded_model.save_model('model.pickle')
        

    elif session_choice.upper() == 'B':
        exchange_years = input("Enter the years (separated by commas): ")
        currency_rates = input("Enter the inflation rates (separated by commas): ")

        years = [int(year.strip()) for year in exchange_years.split(',')]
        rates = [float(rate.strip()) for rate in currency_rates.split(',')]

        for year, rate in zip(years, rates):
            Exchange_loaded_model.process_new_data([[year]], [[rate]])
        Exchange_loaded_model.save_model('exchange_model.pkl')

    elif session_choice.upper() == 'C':
        naira_amount = float(input("Enter the naira amount: "))
        years = int(input("Enter the number of years: "))
        action = input("Do you want to calculate the future value (Enter 'F') or past value (Enter 'P')? ")

        if action.upper() == 'F':
            result = calculate_value(naira_amount, years, future=True)
            print(f"The future value of {naira_amount} naira in {years} years is: {result}")
        elif action.upper() == 'P':
            result = calculate_value(naira_amount, years, future=False)
            print(f"The value of {naira_amount} naira {years} years ago was: {result}")
        else:
            print("Invalid action choice.")

    elif session_choice.upper() == 'E':
        print("Exiting the program...")
        break

    elif session_choice.upper() == 'M':
        continue

    else:
        print("Invalid session choice. Please try again.")
